In [107]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error, r2_score


In [108]:
# Define the dataset class
class EnergyDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = self.X.iloc[idx]
        y = self.y.iloc[idx]
        return X.values, y.values

In [109]:
# Load the data
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_val = pd.read_csv('X_val.csv')
y_val = pd.read_csv('y_val.csv')
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')

In [110]:
# Create the dataset
train_dataset = EnergyDataset(X_train, y_train)
val_dataset = EnergyDataset(X_val, y_val)
test_dataset = EnergyDataset(X_test, y_test)

In [111]:
# Data Loader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [112]:
import torch
import torch.nn as nn

# Define the QCNN model with appropriate activation functions
class QCNN(nn.Module):
    def __init__(self):
        super(QCNN, self).__init__()

        # Input layer(24 features to 256 neurons)
        self.fc1 = nn.Linear(24, 256)    
        self.bn1 = nn.BatchNorm1d(256)    
        self.relu = nn.ReLU()              
        self.dropout = nn.Dropout(0.3)     
        
        # Hidden layer 1(256 to 128 neurons)
        self.fc2 = nn.Linear(256, 128)    
        self.bn2 = nn.BatchNorm1d(128)    
        
        # Hidden layer 2 (128 to 64 neurons)
        self.fc3 = nn.Linear(128, 64)     
        self.bn3 = nn.BatchNorm1d(64)      
        
        # Hidden layer 3 (64 to 32 neurons)
        self.fc4 = nn.Linear(64, 32)      
        self.bn4 = nn.BatchNorm1d(32)      
        
        # Output layer(32 neurons to 1 output)
        self.fc5 = nn.Linear(32, 1)        

    def forward(self, x):
        # Forward pass through the network
        
        #Input Layer
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        # Hidden Layer 
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        x = self.fc3(x)
        x = self.bn3(x)
        x = self.relu(x)

        x = self.fc4(x)
        x = self.bn4(x)
        x = self.relu(x)
        
        # Output Layer
        x = self.fc5(x)
        return x


In [113]:
# Set hyperparameters
learning_rate = 0.001
n_epochs = 200

In [114]:
# Initialize model, optimizer, and loss function
model = QCNN()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

In [115]:
# Lists to log training and validation loss and accuracy
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

In [116]:
# Train the model
for epoch in range(n_epochs):
    model.train()  # Set the model to training mode
    epoch_loss = 0
    for batch in train_loader:
        X, y = batch
        X = X.clone().detach().float()
        y = y.clone().detach().float()  
        optimizer.zero_grad()
        outputs = model(X)
        loss = loss_fn(outputs, y)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()

    # Calculate average loss for the epoch
    average_loss = epoch_loss / len(train_loader)
    train_losses.append(average_loss)

    # Calculate training accuracy (R² score)
    with torch.no_grad():
        y_train_pred = model(torch.tensor(X_train.values, dtype=torch.float32)).flatten().numpy()
        r2 = r2_score(y_train.values, y_train_pred)
        accuracy = r2 * 100
        train_accuracies.append(accuracy)

    # Validate the model
    model.eval()  # Set the model to evaluation mode
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            X, y = batch
            X = X.clone().detach().float()
            y = y.clone().detach().float()  
            outputs = model(X)
            loss = loss_fn(outputs, y)
            val_loss += loss.item()
        
        average_val_loss = val_loss / len(val_loader)
        val_losses.append(average_val_loss)

        y_val_pred = model(torch.tensor(X_val.values, dtype=torch.float32)).flatten().numpy()
        val_r2 = r2_score(y_val.values, y_val_pred)
        val_accuracy = val_r2 * 100
        val_accuracies.append(val_accuracy)

    print(f'Epoch [{epoch + 1}/{n_epochs}], Train Loss: {average_loss}')


Epoch [1/200], Train Loss: 0.008832881310918262
Epoch [2/200], Train Loss: 0.003347938344237119
Epoch [3/200], Train Loss: 0.002807968041878995
Epoch [4/200], Train Loss: 0.0025850736627608993
Epoch [5/200], Train Loss: 0.002453511380892541
Epoch [6/200], Train Loss: 0.002301266304400326
Epoch [7/200], Train Loss: 0.002059259650643825
Epoch [8/200], Train Loss: 0.0019391761179980026
Epoch [9/200], Train Loss: 0.0018715921610610795
Epoch [10/200], Train Loss: 0.0017676994579721539
Epoch [11/200], Train Loss: 0.001704873104199993
Epoch [12/200], Train Loss: 0.0016378758009523153
Epoch [13/200], Train Loss: 0.0016916817726690026
Epoch [14/200], Train Loss: 0.0016171232352262142
Epoch [15/200], Train Loss: 0.0015300253250288887
Epoch [16/200], Train Loss: 0.0015074266755611622
Epoch [17/200], Train Loss: 0.001411894126393015
Epoch [18/200], Train Loss: 0.0013643376766904372
Epoch [19/200], Train Loss: 0.0014510485970486815
Epoch [20/200], Train Loss: 0.0012975824400560986
Epoch [21/200], T

In [117]:
# Save trained model and metrics
torch.save({
    'model_state_dict': model.state_dict(),
    'train_losses': train_losses,
    'train_accuracies': train_accuracies,
    'val_losses': val_losses,
    'val_accuracies': val_accuracies
}, 'models/qcnn_model.pth')

In [118]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Test the model on the test set
model.eval()  # Set the model to evaluation mode
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

with torch.no_grad():
    y_test_pred = model(X_test_tensor).flatten().numpy()
    
    # Calculate MSE, MAE, RMSE, R² score, and MAPE
    test_mse = mean_squared_error(y_test.values, y_test_pred)
    test_r2 = r2_score(y_test.values, y_test_pred)
    test_mae = mean_absolute_error(y_test.values, y_test_pred)
    test_rmse = mean_squared_error(y_test.values, y_test_pred, squared=False)
    test_mape = np.mean(np.abs((y_test.values - y_test_pred) / y_test.values)) * 100

    # Calculate accuracy as R² score in percentage
    test_accuracy = test_r2 * 100
    
    print(f'Mean Squared Error on test data: {test_mse}')
    print(f'R² score (Accuracy) on test data: {test_r2}')
    print(f'Accuracy: {test_accuracy}%')
    print(f'Mean Absolute Error (MAE) on test data: {test_mae}')
    print(f'Root Mean Squared Error (RMSE) on test data: {test_rmse}')
    print(f'Mean Absolute Percentage Error (MAPE) on test data: {test_mape}%')


c:\Users\Asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Mean Squared Error on test data: 0.00045295918787563715
R² score (Accuracy) on test data: 0.9884499426723733
Accuracy: 98.84499426723733%
Mean Absolute Error (MAE) on test data: 0.01524676402903017
Root Mean Squared Error (RMSE) on test data: 0.02128283787176036
Mean Absolute Percentage Error (MAPE) on test data: 86.85117784328412%
